# 2-5. RAG-Vector Store
벡터 저장소(Vector Store)는 벡터 형태로 표현된 데이터, 즉 임베딩 벡터들을 효율적으로 저장하고 검색할 수 있는 시스템이나 데이터베이스를 의미   
- 벡터 저장
- 벡터 검색
- 결과 반환

### 2-5-1. Chroma
- 임베딩 및 메타데이터 저장: 대규모의 임베딩 데이터와 이와 관련된 메타데이터를 효율적으로 저장할 수 있음
- 문서 및 쿼리 임베딩: 텍스트 데이터를 벡터 공간에 매핑하여 임베딩을 생성할 수 있으며, 이를 통해 검색 작업이 가능
- 임베딩 검색: 사용자 쿼리에 기반하여 가장 관련성 높은 임베딩을 찾아내는 검색 기능을 제공

**1. 유사도 기반 검색(Similarity search)**

Chroma 벡터 저장소를 사용하여 대규모 텍스트 데이터셋에서 빠르고 효율적으로 유사도 기반 검색(Similarity search)을 수행

1. 데이터 로드

2. 텍스트 분할
- RecursiveCharacterTextSplitter를 사용하여 로드된 텍스트를 여러 개의 작은 조각으로 분할


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader('history.txt',encoding='utf-8')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)

texts = text_splitter.split_text(data[0].page_content)
texts[0]

'한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.\n\n고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.'

3. 임베딩 모델 초기화 
4. Chroma 벡터 저장소 생성
- Chroma.from_texts 메소드를 사용하여 분할된 텍스트들을 임베딩하고, 이 임베딩을 Chroma 벡터 저장소에 저장

In [2]:
embeddings_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

db = Chroma.from_texts(
    texts,
    embeddings_model,
    collection_name='history',
    persist_directory='./db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'},
)

db

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12500\2394470721.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
c:\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


5. 유사도 기반 검색 수행
- query 변수에 검색 쿼리를 정의
- db.similarity_search 메소드를 사용하여 저장된 데이터 중에서 쿼리와 가장 유사한 문서를 찾음
- 검색 결과를 docs 변수에 저장하고, 가장 유사한 문서의 내용은 docs[0].page_content를 통해 확인

In [3]:
query = '누가 한글을 창제했나요?'
docs = db.similarity_search(query)
print(docs[0].page_content)

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


** 2. MMR(Maximum marginal relevance search)**

최대 한계 관련성(Maximum Marginal Relevance, MMR) 검색 방식은 유사성과 다양성의 균형을 맞추어 검색 결과의 품질을 향상시키는 알고리즘   
검색 쿼리에 대한 문서들의 관련성을 최대화하는 동시에, 검색된 문서들 사이의 중복성을 최소화하여, 사용자에게 다양하고 풍부한 정보를 제공하는 것을 목표   

MMR은 쿼리에 대한 각 문서의 유사성 점수와 이미 선택된 문서들과의 다양성(또는 차별성) 점수를 조합하여, 각 문서의 최종 점수를 계산   
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>MMR</mtext>
  <mo>=</mo>
  <mi>&#x3BB;</mi>
  <mo>&#x22C5;</mo>
  <mtext>Sim</mtext>
  <mo stretchy="false">(</mo>
  <mi>d</mi>
  <mo>,</mo>
  <mi>Q</mi>
  <mo stretchy="false">)</mo>
  <mo>&#x2212;</mo>
  <mo stretchy="false">(</mo>
  <mn>1</mn>
  <mo>&#x2212;</mo>
  <mi>&#x3BB;</mi>
  <mo stretchy="false">)</mo>
  <mo>&#x22C5;</mo>
  <munder>
    <mo data-mjx-texclass="OP" movablelimits="true">max</mo>
    <mrow data-mjx-texclass="ORD">
      <msup>
        <mi>d</mi>
        <mo data-mjx-alternate="1">&#x2032;</mo>
      </msup>
      <mo>&#x2208;</mo>
      <msup>
        <mi>D</mi>
        <mo data-mjx-alternate="1">&#x2032;</mo>
      </msup>
    </mrow>
  </munder>
  <mtext>Sim</mtext>
  <mo stretchy="false">(</mo>
  <mi>d</mi>
  <mo>,</mo>
  <msup>
    <mi>d</mi>
    <mo data-mjx-alternate="1">&#x2032;</mo>
  </msup>
  <mo stretchy="false">)</mo>
</math>

- <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mo stretchy="false">(</mo>
  <mtext>Sim</mtext>
  <mo stretchy="false">(</mo>
  <mi>d</mi>
  <mo>,</mo>
  <mi>Q</mi>
  <mo stretchy="false">)</mo>
  <mo stretchy="false">)</mo>
</math>
는 문서(d)와 쿼리 (Q) 사이의 유사성

- <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mo stretchy="false">(</mo>
  <munder>
    <mo data-mjx-texclass="OP" movablelimits="true">max</mo>
    <mrow data-mjx-texclass="ORD">
      <msup>
        <mi>d</mi>
        <mo data-mjx-alternate="1">&#x2032;</mo>
      </msup>
      <mo>&#x2208;</mo>
      <msup>
        <mi>D</mi>
        <mo data-mjx-alternate="1">&#x2032;</mo>
      </msup>
    </mrow>
  </munder>
  <mtext>Sim</mtext>
  <mo stretchy="false">(</mo>
  <mi>d</mi>
  <mo>,</mo>
  <msup>
    <mi>d</mi>
    <mo data-mjx-alternate="1">&#x2032;</mo>
  </msup>
  <mo stretchy="false">)</mo>
  <mo stretchy="false">)</mo>
</math>는 문서 (d)와 이미 선택된 문서 집합(D') 중 가장 유사한 문서와의 유사성

-<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mo stretchy="false">(</mo>
  <mi>&#x3BB;</mi>
  <mo stretchy="false">)</mo>
</math> 는 유사성과 다양성의 상대적 중요도를 조절하는 매개변수 

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

loader = PyMuPDFLoader('323410_카카오뱅크_2023.pdf')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)
documents = text_splitter.split_documents(data)
len(documents)

c:\langchain\venv\lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:322: UserWarning: Warning: Empty content on page 78 of document 300720_한일시멘트_2023.pdf
  warnings.warn(


229

In [10]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

db2 = Chroma.from_documents(
    documents,
    embeddings_model,
    collection_name='esg',
     persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'},
)

db2

1. 일반적인 유사도 기반 검색:  
- 쿼리 '카카오뱅크의 환경목표와 세부추진내용을 알려줘?'를 사용하여 db2에서 유사성 검색을 수행   
- len(docs)는 반환된 문서의 총 수를 출력   
- docs[0].page_content는 검색 결과 중 가장 유사한 문서의 내용을 출력   

In [ ]:
query = '카카오뱅크의 환경목표와 세부추진내용을 알려줘?'
docs = db2.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

In [ ]:
print(docs[-1].page_content)

2. MMR 검색:   
동일한 쿼리를 사용하여 MMR 검색을 수행   
len(mmr_docs)는 MMR 검색으로 선택된 문서의 총 수, 여기서는 4개를 출력   
mmr_docs[0].page_content는 MMR 검색 결과 중 가장 높은 순위의 문서의 내용을 출력

In [ ]:
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

In [ ]:
print(mmr_docs[-1].page_content)

In [11]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.",
        metadata={
            "title": "LangChain 소개",
            "author": "AI 개발자",
            "url": "http://example.com/langchain-intro"
        }
    ),
    Document(
        page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.",
        metadata={
            "title": "벡터 데이터베이스 개요",
            "author": "데이터 과학자",
            "url": "http://example.com/vector-db-overview"
        }
    )
    # 추가 문서들...
]


In [13]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)


# Chroma 벡터 스토어에 문서와 메타데이터 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # 벡터 스토어를 디스크에 저장
)


In [14]:
query = "LangChain이란 무엇인가요?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(f"내용: {doc.page_content}")
    print(f"제목: {doc.metadata['title']}")
    print(f"저자: {doc.metadata['author']}")
    print(f"URL: {doc.metadata['url']}")
    print("---")


내용: LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.
제목: LangChain 소개
저자: AI 개발자
URL: http://example.com/langchain-intro
---
내용: 벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.
제목: 벡터 데이터베이스 개요
저자: 데이터 과학자
URL: http://example.com/vector-db-overview
---


### 2-5-2. FAISS
FAISS(Facebook AI Similarity Search)는 Facebook AI Research에 의해 개발된 라이브러리로, 대규모 벡터 데이터셋에서 유사도 검색을 빠르고 효율적으로 수행할 수 있게 해줌   
FAISS는 특히 벡터의 압축된 표현을 사용하여 메모리 사용량을 최소화하면서도 검색 속도를 극대화하는 특징 

** 1. 유사도 기반 검색(Similarity search)(Copy) **
- distance_strategy는 벡터 간 거리(또는 유사도)를 측정하는 방법을 결정
- FAISS.from_documents 메서드를 사용하여 문서 객체를 임베딩 벡터로 변환하여 벡터 저장소에 저장

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )
vectorstore

In [18]:
vectorstore.distance_strategy

<DistanceStrategy.COSINE: 'COSINE'>

vectorstore.similarity_search 메서드는 주어진 쿼리 문자열에 대해 벡터 스토어 내의 문서들 중에서 가장 유사한 문서들을 찾아내는 작업을 다음 단계에 따라 수행

1. 쿼리 인코딩: 주어진 쿼리 문자열을 벡터로 변환. 이 과정은 vectorstore를 생성할 때 사용된 임베딩 모델(embeddings_model)을 사용하여 수행
2. 유사도 검색: 변환된 쿼리 벡터와 벡터 스토어에 저장된 문서 벡터들 간의 유사도를 계산하여, 가장 유사한 문서들을 찾아냄. 이 때, 유사도 계산 방법은 vectorstore 생성 시 지정된 distance_strategy에 따라 결정.
3. 결과 반환: 검색 결과로 얻어진 문서들을 유사도 순으로 정렬하여 반환.   

In [19]:
query = '카카오뱅크가 중대성 평가를 통해 도출한 6가지 중대 주제는 무엇인가?'
docs = vectorstore.similarity_search(query)
print(len(docs))
print(docs[0].page_content)


2
LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.


** 2-5-2-2. MMR (Maximum marginal relevance search) (Copy) **

In [ ]:
mmr_docs = vectorstore.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)


** 3. FAISS DB를 로컬에 저장하기 **

In [24]:
# 벡터 스토어 저장하기
vectorstore.save_local('./db/faiss')

In [25]:
# 벡터 스토어 불러오기
db3 = FAISS.load_local('./db/faiss', embedding_model)


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).